# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
while True:
    # Scroll down.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    print('網頁更新中...')
    
    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print('到達頁面底端')
        break

    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 三星新款旗艦機傳將「跳號」取名Galaxy S20　不叫S11可避免與iPhone 11撞號

▲Galaxy S系列新機將可能跳號命名。（圖／取自CashKaro）
記者謝仁傑／綜合報導
三星的S系列新機可能不取名為Galaxy S11？爆料達人 冰宇宙（Ice universe）在推特發文，預測三星新機將直接「跳號」到Galaxy S20。
外界猜測，可能是為了不跟iPhone 11撞號，顯得自己落後，另外也有一說，因為明年為2020，剛好象徵S系列從10跨到20的突破性。
蘋果也曾在新手機命名，出現變動，眾所周知，曾在2017、2018跳過數字，改以「X」命名，不過，部分消費者對此卻感到混淆，弄不清楚誰先誰後，今年， 蘋果還是回歸數字命名，推出iPhone 11系列。
三星Galaxy S系列新機，目前預計將於2020年初發表，綜合先前報導，系列機款中螢幕最大的Galaxy S11+（或是取名S12+）將配備6.9吋螢幕，螢幕更新率有機會達120Hz，外界預期該機款將搭載螢幕下指紋感測及Dynamic AMOLED面板。機身尺寸則為166.9mmx76mmx8.8mm，這較Galaxy S10+更長、更寬，也更厚，而更大的機身也為相機升級預留了空間。
----------------------------------------------------------------------

[1] 瓦城新品牌泰式燒烤落腳信義遠百　挹注年末營收攀漲

▲年末迎來歡聚尾牙，瓦城泰統集團旗下餐廳分別祭出「尾牙春酒旺年宴」套餐搶攻節慶商機。（圖／瓦城提供）
記者林淑慧／台北報導
全台最大東方餐飲連鎖集團瓦城今（25）日表示，旗下第八個新品牌「月月Thai BBQ泰式燒烤」，落腳信義遠百A13的「遠百深夜食堂」，結合湘菜「1010湘食堂」開出雙品牌首店，開幕30分鐘旋即滿座，可望挹注年末營收持續成長。
瓦城泰統集團持續致力於東方餐飲菜系，挾著近30年深厚的廚藝底蘊，創造眾多超高人氣品牌，此次集團全新第八品牌「月月Thai BBQ泰式燒烤」即日起於信義遠百A13搶先公開試營運，搶攻聖誕、跨年聚餐商機。
瓦城主管指出，新品